In [9]:
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [49]:
methods = pd.read_csv("../Data/Voting Methods/Combined/town_meeting_voting_methods_long.csv")

# drop all rows with year 2017 since they use weird categories for voting methods
methods = methods[methods["year"] != 2017]

methods.head()

,town_city,year,method_description,usage,voting_method
2321,ADDISON,2018,Does your town use AUSTRALIAN BALLOT voting fo...,Yes,town_ab_officers
2322,ADDISON,2018,Does your town use AUSTRALIAN BALLOT voting fo...,Yes,town_ab_budget
2323,ADDISON,2018,Does your town use AUSTRALIAN BALLOT voting fo...,Yes,town_ab_public_q
2324,ADDISON,2018,Does your town use AUSTRALIAN BALLOT voting fo...,NaN,town_sd_ab_officers
2325,ADDISON,2018,Does your town use AUSTRALIAN BALLOT voting fo...,NaN,town_sd_ab_budget


In [73]:
# create altair stacked bar chart of how many towns "usage" is "No" for each voting method, grouped by year
chart = alt.Chart(methods).mark_bar().encode(
    x="count()",
    y="voting_method:N",
    tooltip=["year:O", "voting_method:N", "count()"],
    color="usage:N",
    row="year:O"
).properties(
    title="Number of Towns Not Using Each Voting Method by Year"
)
chart 

alt.Chart(...)

In [134]:
# count the number of "Yes" values for "usage" for each "voting_method" and "year"
usage_counts = methods.groupby(["voting_method", "year", "usage"]).size().reset_index(name="count")

# plot usage_counts as a line chart with altair
# facet the plot by "usage" so we can see the trends for each voting method over time
line_chart = alt.Chart(usage_counts, width=500).mark_line().encode(
    x=alt.X("year:O", title="Year"),
    y=alt.Y("count:Q", title="Number of Towns"),
    color=alt.Color("usage:N", title="Usage"),
    row=alt.Row("voting_method:N", title="Voting Method"),
).properties(
    title="Trends in Voting Method Usage Over Time",
    height=200
)
# line_chart

In [135]:
turnout = pd.read_csv("../Data/Turnout Statistics/Combined/town-meeting-turnout-manual.csv", low_memory=False)

# make all strings in "TOWN" uppercase
turnout["TOWN"] = turnout["TOWN"].str.upper()

turnout.head()

,TOWN,Floor Vote Turnout (%),Australian ballot Turnout (%),Absentee ballot Turnout (%),year,aus-due-to-covid-in-2021
0,ALBANY,0.159090909,NaN,NaN,2014,NaN
1,ADDISON,0.094094094,0.325325325,0.061538462,2014,NaN
2,ALBURGH,NaN,0.296627667,0.134570766,2014,NaN
3,ANDOVER,0.210918114,NaN,NaN,2014,NaN
4,ARLINGTON,NaN,0.226500566,0.0775,2014,NaN


In [136]:
# calculate the average turnout for each year and drop rows that are strings
turnout["turnout"] = pd.to_numeric(turnout["Australian ballot Turnout (%)"], errors="coerce")
average_turnout = turnout.groupby("year")["turnout"].mean().reset_index()
average_turnout['type'] = 'avg australian ballot turnout'

# calculate the average turnout for each year and drop rows that are strings
turnout["turnout"] = pd.to_numeric(turnout["Floor Vote Turnout (%)"], errors="coerce")
average_floor_turnout = turnout.groupby("year")["turnout"].mean().reset_index()
average_floor_turnout['type'] = 'avg floor vote turnout'

# calculate the average turnout for each year and drop rows that are strings
turnout["turnout"] = pd.to_numeric(turnout["Absentee ballot Turnout (%)"], errors="coerce")
average_abs_turnout = turnout.groupby("year")["turnout"].mean().reset_index()
average_abs_turnout['type'] = 'avg absentee ballot turnout'
    
average_abs_turnout

,year,turnout,type
0,2014,0.103949,avg absentee ballot turnout
1,2015,0.107703,avg absentee ballot turnout
2,2016,0.125087,avg absentee ballot turnout
3,2017,0.115228,avg absentee ballot turnout
4,2018,0.111325,avg absentee ballot turnout
5,2019,0.109950,avg absentee ballot turnout
6,2020,0.135812,avg absentee ballot turnout
7,2021,0.547681,avg absentee ballot turnout
8,2022,NaN,avg absentee ballot turnout
9,2023,0.154770,avg absentee ballot turnout


In [137]:
# concat vertically the three dataframes
average_turnout_all = pd.concat([average_turnout, average_floor_turnout, average_abs_turnout], ignore_index=True)
average_turnout_all.head()

,year,turnout,type
0,2014,0.331958,avg australian ballot turnout
1,2015,0.238854,avg australian ballot turnout
2,2016,0.470665,avg australian ballot turnout
3,2017,0.278414,avg australian ballot turnout
4,2018,0.240275,avg australian ballot turnout


In [157]:
# use altair to plot line chart
line_chart = alt.Chart(average_turnout_all, width=500).mark_line().encode(
    x=alt.X("year:O", title="Year"),
    y=alt.Y("turnout:Q", title="Average Turnout (%)"),
    color=alt.Color("type:N", title="Turnout Type")
).properties(
    title="Average Turnout Over Time by Turnout Type"
)
line_chart

alt.Chart(...)

In [154]:
vtpop = pd.read_csv("../Data/vt-town-pops.csv")
vtpop["TOWN"] = vtpop["TOWN"].str.upper()

vtpop[vtpop["TOWN"] == "BURLINGTON"]

,TOWN,Total Population,state,county,county subdivision,Census Dataset,Census Year,Total Population (%)
59,BURLINGTON,44675,50,7,10675,ACS 5-Year Estimates,2024,100.0


In [158]:
# filter turnout to only include year 2020
turnout_2019 = turnout[turnout["year"] == 2019]

# merge turnout_2019 with vtpop on "TOWN"
turnout_2019_pop = pd.merge(turnout_2019, vtpop, on="TOWN", how="inner")

turnout_2019_pop.head()

,TOWN,Floor Vote Turnout (%),Australian ballot Turnout (%),Absentee ballot Turnout (%),year,aus-due-to-covid-in-2021,turnout,Total Population,state,county,county subdivision,Census Dataset,Census Year,Total Population (%)
0,ADDISON,NaN,0.225285171,0.063291139,2019,NaN,0.063291,1175,50,1,325,ACS 5-Year Estimates,2024,100.0
1,ALBANY,0.14887218,NaN,NaN,2019,NaN,NaN,934,50,19,475,ACS 5-Year Estimates,2024,100.0
2,ALBURGH,NaN,0.362612613,0.111801242,2019,NaN,0.111801,1832,50,13,860,ACS 5-Year Estimates,2024,100.0
3,ANDOVER,0.143902439,NaN,NaN,2019,NaN,NaN,616,50,27,1300,ACS 5-Year Estimates,2024,100.0
4,ARLINGTON,NaN,0.262378765,0.09922179,2019,NaN,0.099222,2598,50,3,1450,ACS 5-Year Estimates,2024,100.0


In [159]:
# drop row with TOWN of "RUPERT"
turnout_2019_pop = turnout_2019_pop[turnout_2019_pop["TOWN"] != "RUPERT"]

# create altair scatter plot of "Australian ballot Turnout (%)" vs "Total Population"
scatter_plot = alt.Chart(turnout_2019_pop, width=500).mark_circle(size=60).encode(
    x=alt.X("Total Population:Q", title="Total Population"),
    y=alt.Y("Floor Vote Turnout (%):Q", title="Floor Vote Turnout (%)"),
    tooltip=["TOWN:N", "Total Population:Q", "Floor Vote Turnout (%):Q"]
).properties(
    title="Floor Vote Turnout vs Total Population (2019)"
)
scatter_plot

alt.Chart(...)